In [43]:
import time
import pandas as pd
import numpy as np
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [44]:
teacher = ['FK_COD_DOCENTE', 'PK_COD_ENTIDADE','ID_ESPECIALIZACAO','ID_MESTRADO','ID_DOUTORADO']

school = ['ANO_CENSO','PK_COD_ENTIDADE', 'FK_COD_MUNICIPIO','FK_COD_ESTADO','ID_DEPENDENCIA_ADM','ID_AGUA_INEXISTENTE',
                  'ID_ESGOTO_INEXISTENTE','ID_ENERGIA_INEXISTENTE',
                   'ID_LABORATORIO_INFORMATICA','ID_LABORATORIO_CIENCIAS','ID_SALA_ATENDIMENTO_ESPECIAL',
                   'ID_BIBLIOTECA','ID_SALA_LEITURA','ID_SANITARIO_FORA_PREDIO',
                   'ID_SANITARIO_PNE','ID_DEPENDENCIAS_PNE','NUM_SALAS_UTILIZADAS','ID_EQUIP_TV',
                   'ID_EQUIP_DVD','ID_EQUIP_COPIADORA','ID_EQUIP_IMPRESSORA','NUM_COMP_ADMINISTRATIVOS',
                   'NUM_COMP_ALUNOS','ID_INTERNET','ID_BANDA_LARGA','NUM_FUNCIONARIOS','ID_ALIMENTACAO']

enem1 = ['NU_INSCRICAO','NU_ANO','PK_COD_ENTIDADE','IN_TP_ENSINO','IDADE','TP_SEXO','TP_COR_RACA','ST_CONCLUSAO','NU_NT_CN',
        'NU_NT_CH','NU_NT_LC','NU_NT_MT','NU_NOTA_REDACAO'
]

enem2 = ['NU_INSCRICAO','Q01','Q02','Q04','Q03']

enem = ['NU_ANO','PK_COD_ENTIDADE','IN_TP_ENSINO','IDADE','TP_SEXO','TP_COR_RACA','ST_CONCLUSAO','NU_NT_CN',
        'NU_NT_CH','NU_NT_LC','NU_NT_MT','NU_NOTA_REDACAO','Q01','Q02','Q04','Q03'
]


In [45]:
#Students of ENEM
E1 = pd.read_csv('~/data/enem/2012/DADOS/DADOS_ENEM_2012.csv', sep=',', encoding="iso-8859-2", usecols = enem1)
E2 = pd.read_csv('~/data/enem/2012/DADOS/QUESTIONARIO_ENEM_2012.csv', sep=',', usecols = enem2)
E = pd.merge(E1, E2, on = 'NU_INSCRICAO', how= "outer")

In [46]:
#School census data
CE =    pd.read_csv("~/data/censo/2012/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2", usecols = school)

In [47]:
#Teacher census data by Braazilian mesoregion
CD_CO = pd.read_csv("~/data/censo/2012/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_NE = pd.read_csv("~/data/censo/2012/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2",usecols = teacher)
CD_N =  pd.read_csv("~/data/censo/2012/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2",  usecols = teacher)
CD_SE = pd.read_csv("~/data/censo/2012/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2",  usecols = teacher)
CD_S =  pd.read_csv("~/data/censo/2012/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [48]:
ENEM = E.copy()
C_SCHOOL = CE.copy()
C_TEACHER = CT.copy()

In [49]:
#ordering
ENEM = ENEM[enem]

newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

print('checking year',ENEM.CO_ANO.unique())
print('checking key',ENEM.CO_ESCOLA.dtypes)

checking year [2012]
checking key object


In [50]:
C_SCHOOL = C_SCHOOL[school]

newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO_FORA_PREDIO',
 'IN_BANHEIRO_PNE',
 'IN_DEPENDENCIAS_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COM_ADMINISTRATIVO',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO'
]

#Rename 
C_SCHOOL.columns =  newNames

print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_SCHOOL.CO_ESCOLA.dtypes)

Checking year of censo [2012]
checking key int64


In [51]:
C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'CO_PROFESSOR',
'CO_ESCOLA',  
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO'
]
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
C_TEACHER.columns = newNames
C_TEACHER.drop_duplicates(subset=['CO_PROFESSOR', 'CO_ESCOLA'], inplace=True)
C_TEACHER.drop('CO_PROFESSOR', axis=1, inplace = True)

print('checking key',C_TEACHER.CO_ESCOLA.dtypes)

checking key int64


### Minimum Scope Definition

In [52]:
#CO_ESCOLA as ".". It will take off in the scope filter
(ENEM.CO_ESCOLA=='.').sum()/ENEM.CO_ESCOLA.shape[0]

0.7365111598643773

In [53]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])
ENEM = ENEM.loc[~(ENEM.CO_ESCOLA=='.')]
ENEM.CO_ESCOLA = ENEM.CO_ESCOLA.astype('float')


In [54]:
#should care about the nulls into scope filter features?
filters = (['TP_ST_CONCLUSAO', 'TP_ENSINO'])
ENEM = ENEM.loc[~(ENEM.TP_ENSINO=='.')] # because are "." in the column. 
ENEM[filters] = ENEM[filters].astype('int')
ENEM[filters].isnull().sum()
#NO, IT'S LOWER

TP_ST_CONCLUSAO    0
TP_ENSINO          0
dtype: int64

In [55]:
print('drop out',((ENEM.TP_ENSINO!= 1)|(ENEM.TP_ENSINO!= 3)).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[((ENEM.TP_ENSINO== 1)|(ENEM.TP_ENSINO== 3))]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)

drop out 1520780 who not attend regular and prof approachs schools


In [56]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())
    print( 'stranger "."', i , (ENEM[i]== '.').sum())
    print( 'stranger "*"', i , (ENEM[i]== '*').sum())
    
    #, "and", ENEM.I==0)


null: NU_NOTA_CN 0
zero: NU_NOTA_CN 0
stranger "." NU_NOTA_CN 187377
stranger "*" NU_NOTA_CN 0
null: NU_NOTA_CH 0
zero: NU_NOTA_CH 0
stranger "." NU_NOTA_CH 187377
stranger "*" NU_NOTA_CH 0
null: NU_NOTA_LC 0
zero: NU_NOTA_LC 0
stranger "." NU_NOTA_LC 207205
stranger "*" NU_NOTA_LC 0
null: NU_NOTA_MT 0
zero: NU_NOTA_MT 0
stranger "." NU_NOTA_MT 207205
stranger "*" NU_NOTA_MT 0
null: NU_NOTA_REDACAO 0
zero: NU_NOTA_REDACAO 0
stranger "." NU_NOTA_REDACAO 245265
stranger "*" NU_NOTA_REDACAO 0


In [57]:
ENEM = ENEM.loc[~(ENEM[fields] == '.').any(axis=1)]

In [58]:
ENEM[fields] = ENEM[fields].astype('float')

In [59]:
ENEM[fields].dtypes

NU_NOTA_CN         float64
NU_NOTA_CH         float64
NU_NOTA_LC         float64
NU_NOTA_MT         float64
NU_NOTA_REDACAO    float64
dtype: object

In [60]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
ENEM.NU_IDADE = ENEM.NU_IDADE.astype('int32')
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 0
taking off 127853


17    586414
18    317975
19     89065
Name: NU_IDADE, dtype: int64

In [61]:
ENEM[fields].describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,993454.000000,993454.000000,993454.000000,993454.000000,993454.000000
mean,478.216389,526.680258,497.570456,522.438977,519.746712
std,81.028030,83.262026,71.422728,122.168725,147.158486
min,319.300000,302.600000,295.400000,277.200000,0.000000
25%,415.600000,468.900000,450.200000,426.000000,420.000000
50%,469.800000,527.000000,499.400000,516.300000,500.000000
75%,532.000000,583.000000,545.700000,607.700000,620.000000
max,847.600000,874.900000,814.300000,955.200000,1000.000000


### Transform features to default categories

In [62]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
ENEM[fields].isnull().sum()

TP_SEXO            0
TP_COR_RACA        0
EDU_PAI            0
EDU_MAE            0
QT_PESSOAS_CASA    0
RENDA_MENSAL       0
dtype: int64

In [63]:
for i in fields:
    print(i,ENEM[i].value_counts().index)

TP_SEXO Int64Index([1, 0], dtype='int64')
TP_COR_RACA Int64Index([1, 3, 2, 4, 0, 5], dtype='int64')
EDU_PAI Index(['E', 'B', 'C', 'G', 'I', 'D', 'F', 'H', 'A'], dtype='object')
EDU_MAE Index(['E', 'C', 'B', 'G', 'D', 'H', 'F', 'I', 'A'], dtype='object')
QT_PESSOAS_CASA Int64Index([4, 3, 5, 6, 2, 7, 8, 9, 1, 10, 11, 12, 13, 20, 14, 15, 16, 17, 18,
            19],
           dtype='int64')
RENDA_MENSAL Index(['B', 'D', 'C', 'E', 'G', 'F', 'H', 'I', 'J', 'Q', 'L', 'M', 'K', 'N',
       'A', 'O', 'P'],
      dtype='object')


In [64]:
#Order relation, is better for numeric categories.
categorical = ['EDU_PAI', 'EDU_MAE', 'RENDA_MENSAL']
for i in categorical:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes
    

In [65]:
ENEM['EDU_PAI_TEMP'] = ENEM['EDU_PAI']
ENEM['EDU_MAE_TEMP'] = ENEM['EDU_MAE']
ENEM['QT_PESSOAS_CASA_TEMP'] = ENEM['QT_PESSOAS_CASA']
ENEM['RENDA_MENSAL_TEMP'] = ENEM['RENDA_MENSAL']
ENEM['TP_SEXO_TEMP'] = ENEM['TP_SEXO']

In [66]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==8, 'EDU_PAI_TEMP']=0


ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==8, 'EDU_MAE_TEMP']=0


ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=0
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==6, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==7, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']>7, 'QT_PESSOAS_CASA_TEMP']=3

ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=0
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[(ENEM['RENDA_MENSAL']>3) & (ENEM['RENDA_MENSAL']<8), 'RENDA_MENSAL_TEMP']=3
ENEM.loc[(ENEM['RENDA_MENSAL']>7) & (ENEM['RENDA_MENSAL']<13), 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']>12, 'RENDA_MENSAL_TEMP']=5

ENEM.loc[ENEM['TP_SEXO']==1, 'TP_SEXO_TEMP']=0
ENEM.loc[ENEM['TP_SEXO']==0, 'TP_SEXO_TEMP']=1

In [67]:
#back to original columns
ENEM['EDU_PAI'] = ENEM['EDU_PAI_TEMP']
ENEM['EDU_MAE'] = ENEM['EDU_MAE_TEMP']
ENEM['QT_PESSOAS_CASA'] = ENEM['QT_PESSOAS_CASA_TEMP']
ENEM['RENDA_MENSAL'] = ENEM['RENDA_MENSAL_TEMP']
ENEM['TP_SEXO'] = ENEM['TP_SEXO_TEMP']


In [68]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP', 'TP_SEXO_TEMP'], axis=1, inplace=True)

In [69]:
print('Total Students at ENEM:', ENEM.shape[0], 'of', ENEM.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
print('Total Schools at Census:', C_SCHOOL.shape[0])

       
#######################################
print('########### Only about Schools at ENEM:')

schoolsEnem = ENEM['CO_ESCOLA'].dropna().unique()
schoolsCenso = C_SCHOOL['CO_ESCOLA'].dropna().unique()
schoolsTeacher = C_TEACHER['CO_ESCOLA'].dropna().unique()
schoolsEnem2 = ENEM['CO_ESCOLA'].dropna()

enem_censo = np.setdiff1d(schoolsEnem ,schoolsCenso)
print('Schools at ENEM out of School Census', sum(np.isin(schoolsEnem, schoolsCenso, invert=True)))

print('Schools at ENEM out of Teacher Census', sum(np.isin(schoolsEnem, schoolsTeacher, invert=True)))
print('Schools at ENEM and School Census out of Teacher Census', sum(np.isin(enem_censo, schoolsTeacher)))

print('Students Wasted*****', sum(np.isin(schoolsEnem2, schoolsCenso, invert=True)))

Total Students at ENEM: 993454 of 27013 schools
Total Schools at Census: 262601
########### Only about Schools at ENEM:
Schools at ENEM out of School Census 198
Schools at ENEM out of Teacher Census 605
Schools at ENEM and School Census out of Teacher Census 0
Students Wasted***** 1807


### New Features

ENEM

In [70]:
#Final Media
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5

SCHOOL CENSUS - Itś better include after merge all datasets years.


TEACHER CENSUS

In [71]:
C_TEACHER.isnull().sum()

CO_ESCOLA            0
IN_ESPECIALIZACAO    0
IN_MESTRADO          0
IN_DOUTORADO         0
dtype: int64

In [72]:
print('number of teacher who work at an school listed at ENEM and do not have study level assigned:',
      C_TEACHER['CO_ESCOLA'].isin(ENEM['CO_ESCOLA']).sum())


#lets fill with lower level

C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

number of teacher who work at an school listed at ENEM and do not have study level assigned: 966494


In [73]:
C_TEACHER['TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

#Rebuild level education information as indicator of unique level.
C_TEACHER['NU_GRADUACAO'] = 0
C_TEACHER['NU_ESPECIALIZACAO'] = 0
C_TEACHER['NU_MESTRADO'] = 0
C_TEACHER['NU_DOUTORADO'] = 0

C_TEACHER.loc[C_TEACHER.TITULACAO == 0, 'NU_GRADUACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 1, 'NU_ESPECIALIZACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 2, 'NU_MESTRADO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 3, 'NU_DOUTORADO']= 1


C_TEACHER['NU_GRADUACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_GRADUACAO'].transform(sum)
C_TEACHER['NU_ESPECIALIZACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_ESPECIALIZACAO'].transform(sum)
C_TEACHER['NU_MESTRADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_MESTRADO'].transform(sum)
C_TEACHER['NU_DOUTORADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_DOUTORADO'].transform(sum)



#Calculating weighted indice of teacher education
SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)  

##arranjar isso dai



#transformin to school grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

## STUDENT GRAIN 

In [74]:
# teacher to Schoool Grain
print('C_SCHOOL',C_SCHOOL.shape)
print('C_TEACHER',C_TEACHER.shape)
CENSO19 = pd.merge(C_SCHOOL, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('C_SCHOOL + C_TEACHER',CENSO19.shape)

#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO_19 = pd.merge(ENEM, CENSO19, on = 'CO_ESCOLA', how = 'inner')
print('FINAL',ENEM_CENSO_19.shape)

ENEM_CENSO_19.to_csv('STUDENT.csv')

C_SCHOOL (262601, 26)
C_TEACHER (194695, 9)
C_SCHOOL + C_TEACHER (194695, 34)
ENEM (993454, 16)
FINAL (989620, 49)


In [75]:
ENEM_CENSO_19.head()

,CO_ANO,CO_ESCOLA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO
0,2012,21161321.0,17,1,3,2,371.1,372.3,321.6,318.9,400.0,1,1,2,2,356.78,2106607,21,2,0.0,0.0,0.0,1,0,0,1,0,0,0,0,9.0,1,1,1,1,3.0,20.0,1.0,1.0,72.0,1.0,0,0,0,0.037037,32,4,0,0
1,2012,21161321.0,18,0,3,2,470.5,405.7,337.7,349.9,300.0,2,1,2,1,372.76,2106607,21,2,0.0,0.0,0.0,1,0,0,1,0,0,0,0,9.0,1,1,1,1,3.0,20.0,1.0,1.0,72.0,1.0,0,0,0,0.037037,32,4,0,0
2,2012,21161321.0,18,1,3,2,403.8,360.5,460.2,503.1,480.0,3,3,2,2,441.52,2106607,21,2,0.0,0.0,0.0,1,0,0,1,0,0,0,0,9.0,1,1,1,1,3.0,20.0,1.0,1.0,72.0,1.0,0,0,0,0.037037,32,4,0,0
3,2012,21161321.0,19,0,3,2,404.7,351.3,384.4,318.8,220.0,1,1,2,1,335.84,2106607,21,2,0.0,0.0,0.0,1,0,0,1,0,0,0,0,9.0,1,1,1,1,3.0,20.0,1.0,1.0,72.0,1.0,0,0,0,0.037037,32,4,0,0
4,2012,21161321.0,19,1,3,2,468.6,372.4,338.8,332.0,360.0,2,0,2,2,374.36,2106607,21,2,0.0,0.0,0.0,1,0,0,1,0,0,0,0,9.0,1,1,1,1,3.0,20.0,1.0,1.0,72.0,1.0,0,0,0,0.037037,32,4,0,0


## SCHOOL GRAIN

In [76]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL', 'NU_IDADE', 'NU_NOTA_CN',
             'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','NU_NOTA_GERAL' ]

for i in fields:   
    
            ENEM[i] = ENEM.groupby('CO_ESCOLA')[i].transform('mean')
            

In [77]:
# SCHOOL GRAIN AND SHOW THE NEW NOTA DISTRIBUTION
ENEM.drop_duplicates('CO_ESCOLA', inplace=True)
#print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

In [78]:
#recalculating target for new  Nota Geral distribution 
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

75.00092548032428 % lowers quartis


In [79]:
print('censo before',CENSO19.shape)
print('enem_school before', ENEM.shape)
CENSO_ENEM_19 = pd.merge(CENSO19, ENEM, on='CO_ESCOLA', how='inner')
print(CENSO_ENEM_19.shape)

CENSO_ENEM_19.to_csv('SCHOOL.csv', index=False)


censo before (194695, 34)
enem_school before (27013, 17)
(26408, 50)


In [80]:
CENSO_ENEM_19.head()

,CO_ESCOLA,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO,CO_ANO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,TARGET
0,14005360,1400472,14,2,0.0,0.0,0.0,1,0,1,1,0,0,0,0,10.0,1,1,0,1,6.0,10.0,1.0,0.0,57.0,1.0,0,0,0,0.057971,19,4,0,0,2012,17.333333,0.333333,2.000000,2,466.766667,511.266667,515.733333,506.400000,573.333333,2.333333,2.666667,2.000000,2.666667,514.700000,0
1,14000814,1400100,14,4,0.0,0.0,0.0,1,1,0,1,0,0,1,0,21.0,1,1,1,1,22.0,59.0,1.0,1.0,55.0,0.0,0,0,0,0.027027,34,3,0,0,2012,17.428571,0.314286,2.085714,2,534.505714,570.488571,520.377143,603.720000,591.428571,3.485714,3.828571,1.714286,4.028571,564.104000,1
2,14324636,1400100,14,4,0.0,0.0,0.0,1,0,1,1,0,1,1,1,41.0,1,1,1,1,25.0,58.0,1.0,1.0,125.0,0.0,0,0,0,0.106996,57,22,2,0,2012,17.204545,0.454545,2.136364,2,500.375000,558.772727,525.475000,585.268182,546.363636,3.000000,3.590909,1.818182,2.977273,543.250909,1
3,14000776,1400100,14,2,0.0,0.0,0.0,1,0,1,1,1,0,0,0,10.0,1,1,1,1,4.0,11.0,1.0,1.0,100.0,1.0,1,0,0,0.176471,16,18,0,0,2012,17.750000,0.375000,2.562500,2,451.931250,502.518750,464.112500,476.143750,452.500000,1.562500,2.500000,1.875000,2.187500,469.441250,0
4,14000490,1400100,14,2,0.0,0.0,0.0,1,1,0,1,1,0,1,0,30.0,1,1,0,1,8.0,38.0,1.0,1.0,130.0,1.0,0,0,0,0.181818,31,34,1,0,2012,17.660793,0.374449,2.422907,2,445.065639,499.637004,472.570485,474.145374,488.898678,2.162996,2.784141,1.788546,2.185022,476.063436,0


In [81]:
CENSO_ENEM_19.isnull().sum()


CO_ESCOLA                          0
CO_MUNICIPIO                       0
CO_UF                              0
CO_DEPENDENCIA_ADM                 0
IN_AGUA_INEXISTENTE                0
IN_ESGOTO_INEXISTENTE              0
IN_ENERGIA_INEXISTENTE             0
IN_LABORATORIO_INFORMATICA         0
IN_LABORATORIO_CIENCIAS            0
IN_SALA_ATENDIMENTO_ESPECIAL       0
IN_BIBLIOTECA                      0
IN_SALA_LEITURA                    0
IN_BANHEIRO_FORA_PREDIO            0
IN_BANHEIRO_PNE                    0
IN_DEPENDENCIAS_PNE                0
QT_SALAS_UTILIZADAS                0
QT_EQUIP_TV                        0
QT_EQUIP_DVD                       0
QT_EQUIP_COPIADORA                 0
QT_EQUIP_IMPRESSORA                0
QT_COM_ADMINISTRATIVO            476
QT_COMP_ALUNO                   1941
IN_INTERNET                      338
IN_BANDA_LARGA                  1145
QT_FUNCIONARIOS                    0
IN_ALIMENTACAO                     0
IN_ESPECIALIZACAO                  0
I